# Data 604: E-Commerce Supply Chain Model

## Objective:
 ### Develop a model that illustrates the process where a product is dispatched from a warehouse and sent to the customer, after purchasing the product through an e-commerce website.


In this project, we will create a model for a e-commerce fulfillment process. It begins with an order being created, processed at a warehouse, then picked up and delivered by a carrier. Adjusting the number of warehouses, number of workers per warehouse, and the number of carriers available will impact the average processing and shipping times of the orders.

In [ ]:
!pip install simpy

First, we import the libraries we will need.

In [ ]:
import simpy
import random

Here, we initialize the simulation environemnt and create a class object ECommerceSimulation. Notice how we create a simply.Resource instances for warehouses and carries because each warehouse has mutliple workers, while each carrier is a single delivery resourse. Also, we want to keep track of the process and shipping times  as well as the number of orders being handled.

The next function we create is the process_order where this simulates an order being processed and packed at a warehouse, then ready to be delivered by a carrier. Using, yield.self.env.timeout(processing_time) to simulate processing delays.

dispatch_order function is called after the orders are processed and then sent to a carrier which will deliver the package to the customers. This function also keeps track of the number of order status that are out-of-delivery, number of order handled by each carrier.

The last function is this class object is handle_order where it uses the other two functions to complete an order process. When receiving an order it would send it to a warehouse for processing and then to a carrier to delivery.

In [ ]:
class ECommerceModel:
    def __init__(self, env, num_warehouses, workers_per_warehouse, num_carriers):
        self.env = env
        self.warehouses = [simpy.Resource(env, capacity=workers_per_warehouse) for _ in range(num_warehouses)]
        self.carriers = [simpy.Resource(env, capacity=1) for _ in range(num_carriers)]
        self.warehouse_times = {i: [] for i in range(num_warehouses)}
        self.carrier_times = {i: [] for i in range(num_carriers)}
        self.warehouse_order_counts = {i: 0 for i in range(num_warehouses)}
        self.carrier_order_counts = {i: 0 for i in range(num_carriers)}
        self.orders_in_progress = 0
        self.orders_out_for_delivery = 0

    def process_order(self, warehouse_id, order_id):
        with self.warehouses[warehouse_id].request() as request:
            yield request
            self.orders_in_progress += 1
            self.warehouse_order_counts[warehouse_id] += 1
            start_time = self.env.now
            processing_time = random.expovariate(1 / 3)
            yield self.env.timeout(processing_time)
            end_time = self.env.now
            self.warehouse_times[warehouse_id].append(end_time - start_time)
            self.orders_in_progress -= 1

    def dispatch_order(self, order_id):
        carrier_id = random.randint(0, len(self.carriers) - 1)
        with self.carriers[carrier_id].request() as request:
            yield request
            self.orders_out_for_delivery += 1
            self.carrier_order_counts[carrier_id] += 1
            start_time = self.env.now
            delivery_time = random.expovariate(1 / 2)
            yield self.env.timeout(delivery_time)
            end_time = self.env.now
            self.carrier_times[carrier_id].append(end_time - start_time)
            self.orders_out_for_delivery -= 1

    def handle_order(self, order_id):
        warehouse_id = random.randint(0, len(self.warehouses) - 1)
        yield self.env.process(self.process_order(warehouse_id, order_id))
        yield self.env.process(self.dispatch_order(order_id))



Now, we need a way to generate orders, then the order get passed into the handle_order function for processing. Using a exponential distribution to generate syntethic data when order are created unpredictably over time.

In [ ]:
def generate_orders(env, simulation, order_interval):
    order_id = 1
    while True:
        yield env.timeout(random.expovariate(1.0 / order_interval))
        env.process(simulation.handle_order(order_id))
        order_id += 1

Next is to create our simulation envirnment with simpy using simpy.Environment(). We use env.process() to keep generating new ordes for as long as the simulation is running similar to how we use 'source' is Simio. For now, we have set the simulation to duration for 1 week or 7 days. env.run() takes a time units parameter where a time unit is one hour.

After running the simulation, we then print our results such as average shipping time, average processing time, and how many order were handled.

In [ ]:
def run_sim(num_warehouses, workers_per_warehouse, num_carriers, order_interval):

    env = simpy.Environment()
    simulation = ECommerceModel(env, num_warehouses, workers_per_warehouse, num_carriers)
    env.process(generate_orders(env, simulation, order_interval))
    simulation_duration = 24 * 7
    env.run(until=simulation_duration)

    print("\n--- Simulation Results ---")

    def print_summary(title, stats):
        print(f"\n{title}")
        for entity_id, times in stats['times'].items():
            avg_time = sum(times) / len(times) if times else 0
            total_orders = stats['counts'][entity_id]
            print(f"Entity {entity_id}: Average Time = {avg_time:.2f}, Total Orders = {total_orders}")

    overall_processing_time = sum([sum(times) for times in simulation.warehouse_times.values()])
    total_processed_orders = sum(simulation.warehouse_order_counts.values())
    avg_overall_processing_time = overall_processing_time / total_processed_orders if total_processed_orders > 0 else 0
    print(f"Overall average processing time: {avg_overall_processing_time:.2f} hours")

    overall_shipping_time = sum([sum(times) for times in simulation.carrier_times.values()])
    total_shipped_orders = sum(simulation.carrier_order_counts.values())
    avg_overall_shipping_time = overall_shipping_time / total_shipped_orders if total_shipped_orders > 0 else 0
    print(f"Overall average shipping time: {avg_overall_shipping_time:.2f} days")


    overall_orders_count = total_processed_orders
    print(f"Overall total orders handled: {overall_orders_count}")

    print(f"Orders still being processed: {simulation.orders_in_progress}")
    print(f"Orders out for delivery: {simulation.orders_out_for_delivery}")

    print_summary("Warehouse Summary", {
        'times': simulation.warehouse_times,
        'counts': simulation.warehouse_order_counts
    })

    print_summary("Carrier Summary", {
        'times': simulation.carrier_times,
        'counts': simulation.carrier_order_counts
    })


Finally, to run the model with the inputs we want to test. Suppose, we have a ecommerce business that has 4 carriers, 3 warehouses, and at each warehouse there exists 5 workers. Note, carriers are the delivery organizations such as USPS, FedEx, or UPS. Some companies like amazon have their own delivery system.

In [ ]:
run_sim(num_warehouses=3, workers_per_warehouse=5, num_carriers=4, order_interval=0.2)



--- Simulation Results ---
Overall average processing time: 3.07 hours
Overall average shipping time: 1.94 days
Overall total orders handled: 771
Orders still being processed: 15
Orders out for delivery: 4

Warehouse Summary
Entity 0: Average Time = 3.24, Total Orders = 240
Entity 1: Average Time = 3.06, Total Orders = 270
Entity 2: Average Time = 3.10, Total Orders = 261

Carrier Summary
Entity 0: Average Time = 2.13, Total Orders = 77
Entity 1: Average Time = 1.72, Total Orders = 93
Entity 2: Average Time = 2.14, Total Orders = 78
Entity 3: Average Time = 1.91, Total Orders = 86


With these parameters, where there are 4 carriers, 3 warehouses and each warehouse has 5 workers, we observe that the overall processing time from order creation to package packed for delivery is 3.07 hours and the overall shipping time is 1.94 days. Below, is the breakdown of how each of warehouses and carriers performed.

Now, what if we double the workforce and set the number of workers to 10. We get,

In [ ]:
run_sim(num_warehouses=3, workers_per_warehouse=10, num_carriers=4, order_interval=0.2)


--- Simulation Results ---
Overall average processing time: 2.94 hours
Overall average shipping time: 1.86 days
Overall total orders handled: 847
Orders still being processed: 13
Orders out for delivery: 4

Warehouse Summary
Entity 0: Average Time = 3.11, Total Orders = 256
Entity 1: Average Time = 3.05, Total Orders = 292
Entity 2: Average Time = 2.81, Total Orders = 299

Carrier Summary
Entity 0: Average Time = 2.11, Total Orders = 79
Entity 1: Average Time = 1.99, Total Orders = 83
Entity 2: Average Time = 1.66, Total Orders = 100
Entity 3: Average Time = 1.82, Total Orders = 91


With double the workers, we were able to reduce overall average time and increase the number of orders to 847 orders completed in a span of 7 days.

In this project, we developed and demonstrated the application of a E-commerce product fulfillment process where orders are created from customers purchased then fulfilled in a warehouse and then shipped to customer's address using a carrier. We can manipulate the inputs to observe how the number of orders handled and average times changed. From this, we can find the best parameters for a given rate of orders with order_interval property.



References:

https://realpython.com/simpy-simulating-with-python/

https://www.crbgroup.com/insights/consulting/warehouse-optimization